In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

In [ ]:
data = pd.read_csv('../input/stroke-prediction-dataset/healthcare-dataset-stroke-data.csv')


In [ ]:
data.columns

# 각 칼럼에 대한 정보 개요
# id ~ 그냥 ID 필요한가?
# genger ~ 성별
# age ~ 나이
# hypertension ~ 고혈압 유무 
# heart_disease ~ 심장병
# ever_married ~ 결혼 유무 ever_married 결혼 유무 ~ yes or no 인데 1,0으로 바꿀 필요
# work_type ~ 총 다섯개
# residenc_type ~ 주거지역, 도시 or 교외
# avg_glucose_level ~ 혈당
# bmi ~ bmi
# smoking_status ~ 흡연유무 , 총 네개

# stroke ~ 고혈압 유무 ~ 결과

In [ ]:
print(data.shape[0]-data.count())

data.shape

# 총 5110에 관한 데이터, 누락값은 bmi에서 201개 ~ 그냥 드랍하자  

data = data.dropna(axis = False)

data.shape

학습에 고려될 데이터 및 EDA

In [ ]:
# 성별에 관하여

data['gender'].value_counts()

# other에 대하여 학습에 방해가 될 가능성 있음, 삭제 필요

drop = data.loc[data['gender'] == 'Other'].index # 조건으로 나오는 행의  인덱스를 저장

data = data.drop(drop) # 그대로 drop함수에 지정해서 해당 인덱스 삭제

print(data['gender'].value_counts()) 
print(data['stroke'].groupby(data['gender']).mean())
# 성별의 분포는 여성쪽이 더 많지만 뇌졸증 발병은 남자가 더 많다. ~ 학습에 유의미

In [ ]:
# 나이에 관하여

print(data['age'].min(), data['age'].max()) # 각 최소 최대값

data['age'].groupby(data['stroke']).mean()
# 뇌졸증 환자의 평균 나이가 더 많음

In [ ]:
# 고혈압에 대하여

data['stroke'].groupby(data['hypertension']).mean() # 고혈압이 없을경우의 뇌졸증 확률 0.03 고혈압 잇을경우 0.13
data['hypertension'].groupby(data['stroke']).mean() # 뇌졸증이 없을경우의 고혈압 확률 0.08 뇌졸증 잇을경우 0.28

# 고혈압이 있을경우 좀 더 많음

In [ ]:
#  심장병에 대하여

data['stroke'].groupby(data['heart_disease']).mean() #심장병이 없을경우의 뇌졸증 확률 0.03 심장병 잇을경우 0.16
data['heart_disease'].groupby(data['stroke']).mean() #뇌졸증이 없을경우의 심장병 확률 0.04 뇌졸증 잇을경우 0.19

# 크게 영향이 잇는가?

# 지병에 대해 크게 영향이 없다고 판단, 그러나 아주 못쓸 요소는 아닌듯... .상식선에서서 심장병에 대하여

data['stroke'].groupby(data['heart_disease']).mean() #심장병이 없을경우의 뇌졸증 확률 0.03 심장병 잇을경우 0.16
data['heart_disease'].groupby(data['stroke']).mean() #뇌졸증이 없을경우의 심장병 확률 0.04 뇌졸증 잇을경우 0.19

# 크게 영향이 잇는가?

# 지병에 대해 크게 영향이 없다고 판단, 그러나 아주 못쓸 요소는 아닌듯... .상식선에서서

In [ ]:
# 결혼 유무에 대하여 ~ yes를 1로 no를 0으로 바꿀 필요

marry = {'Yes': 1, 'No': 0}
 
data['ever_married'] = data['ever_married'].replace(marry)

data['ever_married'].groupby(data['stroke']).mean() 
# 결혼을 한 사람의 뇌졸증 발병이 더 많음

In [ ]:
# 일하는 직종에 대하여

data['work_type'].value_counts()

data['stroke'].groupby(data['work_type']).mean()

# 그나마 유의미하게 수치가 비교되는 듯? 

In [ ]:
# 주거지역에 대하여

data['Residence_type'].value_counts()

data['stroke'].groupby(data['Residence_type']).mean()

# 의미가 거의 없나?

In [ ]:
# 혈당에 대하여

data['avg_glucose_level']

data['avg_glucose_level'].groupby(data['stroke']).mean()

# 뇌졸증 환자의 혈당이 유의미하게 높음음

In [ ]:
# bmi에 대하여

data['bmi']

data['bmi'].groupby(data['stroke']).mean()
# 근소하게 뇌졸증 환자의 bmi가 높음

In [ ]:
# 흡연에 대하여

data['smoking_status']

print(data['stroke'].groupby(data['smoking_status']).mean())

# 의외의 결과,,, 금연자의 확률이 더 높다


# 뇌졸증 환자와 그렇지 않은 자들의 흡연 비율을 한번 살펴보자,, 데이터에 대한 결과가 좀 믿음직 스럽지 못해서

sample = data['stroke']==1
print(data[sample].groupby(data['smoking_status']).count())

sample1 = data['stroke'] ==0
print(data[sample1].groupby(data['smoking_status']).count())

# 어쨋든 비율이 이상한건 아닌듯하다

결국 모든 요소를 학습에 이용할만 하긴 한거같다. 이후 학습진행, 모델 선정

In [ ]:
y_data = data[['stroke']]
x_data = data.drop('stroke', axis=1) 
# 학습데이터와 결과 데이터 나누기


# 원핫인코딩 ~ 워크타입, 사는 곳, 흡연유무가 원핫인코딩 필요
x_data = pd.get_dummies(x_data, columns = ['work_type'])
x_data = pd.get_dummies(x_data, columns = ['Residence_type'])
x_data = pd.get_dummies(x_data, columns=['smoking_status'])
x_data = pd.get_dummies(x_data, columns=['gender'])
print(x_data.shape, y_data.shape)

x_data.dtypes

In [ ]:
X = tf.keras.layers.Input(shape=[20]) #종속변수가 19개
H = tf.keras.layers.Dense(15, activation='swish')(X)
H = tf.keras.layers.Dense(7, activation='swish')(H)
H = tf.keras.layers.Dense(4, activation='swish')(H)
Y = tf.keras.layers.Dense(1, activation='sigmoid')(H) #결과는 단 하나

model = tf.keras.Model(X,Y)

model.compile(loss='mse', optimizer='adam' ,metrics='accuracy')

# 모델 구성, 우선은 타이타닉 때 모델을 그대로 가져오기

In [ ]:
model.fit(x_data, y_data, epochs=100, verbose=1)